In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd /content/drive/MyDrive/Spring\'23/CSCE670/Animendations/
!ls

/content/drive/MyDrive/Spring'23/CSCE670/Animendations
'Animendations: Project Preview.gslides'
 animes_synopsis_embeddings
 data
 Gojo
 preprocessed_data
'Project Proposal: Animendations.gdoc'
'Project Proposal: <Insert Team Name>.gdoc'
 synopsis_autoencoder
'Untitled spreadsheet.gsheet'
 user_profile_dict
 visual_embeddings


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

animes_data_path = "./preprocessed_data/new_preprocessed_animes.csv"
profiles_data_path = "./preprocessed_data/preprocessed_profiles.csv"
reviews_data_path = "./preprocessed_data/new_preprocessed_reviews.csv"

In [3]:
profiles_data = pd.read_csv(profiles_data_path)
reviews_data = pd.read_csv(reviews_data_path)

In [4]:
profiles = profiles_data.copy()
reviews = reviews_data.copy()

## Splitting Review Data

In [5]:
from scipy.sparse import coo_matrix
# import torch
# import torch.nn as nn
import numpy as np

np.random.seed(0)
unique_AnimeID = reviews['anime_uid'].unique()
unique_users = reviews['profile'].unique()
j = 0
user_old2new_id_dict = dict()
user_new2old_id_dict = dict()
for u in unique_users:
    user_old2new_id_dict[u] = j
    user_new2old_id_dict[j] = u
    j += 1

j = 0
movie_old2new_id_dict = dict()
movie_new2old_id_dict = dict()
for i in unique_AnimeID:
    movie_old2new_id_dict[i] = j
    movie_new2old_id_dict[j] = i
    j += 1
    
### Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = reviews['profile'].values
movie_list = reviews['anime_uid'].values
for j in range(len(reviews)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
reviews['profile'] = user_list
reviews['anime_uid'] = movie_list

### generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
train_index = np.random.random(len(reviews)) <= 0.7
train_df = reviews[train_index]
test_df = reviews[~train_index]

### generate train_mat and test_mat
num_user = len(reviews['profile'].unique())
num_items = len(reviews['anime_uid'].unique())

train_mat = coo_matrix((train_df['score'].values, (train_df['profile'].values, train_df['anime_uid'].values)), shape=(num_user, num_items)).astype(float).toarray()
test_mat = coo_matrix((test_df['score'].values, (test_df['profile'].values, test_df['anime_uid'].values)), shape=(num_user, num_items)).astype(float).toarray()
# train_data = torch.FloatTensor(train_mat)
# test_data = torch.FloatTensor(test_mat)

In [6]:
train_df['old_profile'] = train_df.apply(lambda x: user_new2old_id_dict[x['profile']], axis = 1)
test_df['old_profile'] = test_df.apply(lambda x: user_new2old_id_dict[x['profile']], axis = 1)
train_df.head()

<ipython-input-6-c8296660fd54>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['old_profile'] = train_df.apply(lambda x: user_new2old_id_dict[x['profile']], axis = 1)
<ipython-input-6-c8296660fd54>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['old_profile'] = test_df.apply(lambda x: user_new2old_id_dict[x['profile']], axis = 1)


,uid,profile,anime_uid,text,score,scores,link,old_profile
0,255938,0,0,"first things first. my ""reviews"" system is exp...",8,"{'Sound': '10', 'Overall': '8', 'Animation': '...",https://myanimelist.net/reviews.php?id=255938,DesolatePsyche
2,253664,2,2,"art 9/10: it is great, especially the actions ...",7,"{'Story': '7', 'Sound': '8', 'Overall': '7', '...",https://myanimelist.net/reviews.php?id=253664,skrn
3,8254,3,3,taking place 1 yr from where season 1 trailed ...,9,"{'Overall': '9', 'Story': '9', 'Enjoyment': '9...",https://myanimelist.net/reviews.php?id=8254,edgewalker00
4,291149,4,4,kyoto animations greatest strength is being ab...,10,"{'Story': '10', 'Sound': '9', 'Overall': '10',...",https://myanimelist.net/reviews.php?id=291149,aManOfCulture99
5,10046,5,3,when i first saw code geass i was like omgzxwt...,10,"{'Story': '9', 'Animation': '10', 'Sound': '10...",https://myanimelist.net/reviews.php?id=10046,eneri


In [7]:
train_df['old_anime_uid'] = train_df.apply(lambda x: movie_new2old_id_dict[x['anime_uid']], axis = 1)
test_df['old_anime_uid'] = test_df.apply(lambda x: movie_new2old_id_dict[x['anime_uid']], axis = 1)
train_df.head()

<ipython-input-7-8a633eb1175a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['old_anime_uid'] = train_df.apply(lambda x: movie_new2old_id_dict[x['anime_uid']], axis = 1)
<ipython-input-7-8a633eb1175a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['old_anime_uid'] = test_df.apply(lambda x: movie_new2old_id_dict[x['anime_uid']], axis = 1)


,uid,profile,anime_uid,text,score,scores,link,old_profile,old_anime_uid
0,255938,0,0,"first things first. my ""reviews"" system is exp...",8,"{'Sound': '10', 'Overall': '8', 'Animation': '...",https://myanimelist.net/reviews.php?id=255938,DesolatePsyche,34096
2,253664,2,2,"art 9/10: it is great, especially the actions ...",7,"{'Story': '7', 'Sound': '8', 'Overall': '7', '...",https://myanimelist.net/reviews.php?id=253664,skrn,28891
3,8254,3,3,taking place 1 yr from where season 1 trailed ...,9,"{'Overall': '9', 'Story': '9', 'Enjoyment': '9...",https://myanimelist.net/reviews.php?id=8254,edgewalker00,2904
4,291149,4,4,kyoto animations greatest strength is being ab...,10,"{'Story': '10', 'Sound': '9', 'Overall': '10',...",https://myanimelist.net/reviews.php?id=291149,aManOfCulture99,4181
5,10046,5,3,when i first saw code geass i was like omgzxwt...,10,"{'Story': '9', 'Animation': '10', 'Sound': '10...",https://myanimelist.net/reviews.php?id=10046,eneri,2904


In [8]:
test_df.head()

,uid,profile,anime_uid,text,score,scores,link,old_profile,old_anime_uid
1,259117,1,1,let me start off by saying that made in abyss ...,10,"{'Story': '10', 'Animation': '10', 'Sound': '1...",https://myanimelist.net/reviews.php?id=259117,baekbeans,34599
7,23791,7,3,code geass is one of those series that everybo...,10,"{'Story': '10', 'Sound': '10', 'Overall': '10'...",https://myanimelist.net/reviews.php?id=23791,jchang,2904
8,116775,8,6,this story was… well confusing. the problem wa...,8,"{'Enjoyment': '7', 'Animation': '7', 'Story': ...",https://myanimelist.net/reviews.php?id=116775,angelsreview,4672
10,181043,10,4,to begin with clannad is a very peculiar anime...,10,"{'Story': '10', 'Sound': '9', 'Overall': '10',...",https://myanimelist.net/reviews.php?id=181043,Kiethol,4181
13,262977,13,8,an anime brimmed with action and flashy fights...,8,"{'Story': '7', 'Sound': '8', 'Animation': '10'...",https://myanimelist.net/reviews.php?id=262977,OVERPOWERED99,30276


## Creating user-profile

In [9]:
import pickle

# visual_embeddings_path = "./visual_embeddings/compressed_vit_50_mapping_embeddings.pickle"
visual_embeddings_path = "./visual_embeddings/dict_1000_vit.pickle"
with open(visual_embeddings_path, 'rb') as f:
  visual_embeddings = pickle.load(f)
print(visual_embeddings.keys())

dict_keys([1035, 2495, 687, 16668, 3371, 6336, 15809, 7805, 6736, 9837, 5251, 1107, 36655, 136, 6994, 1498, 4163, 11255, 5356, 34544, 334, 2559, 7768, 12231, 2501, 1538, 3638, 1442, 37973, 5348, 987, 21067, 31867, 81, 743, 28249, 4535, 34723, 1377, 2755, 10294, 35974, 1253, 10077, 30394, 10217, 121, 23249, 30155, 1734, 95, 7193, 14833, 644, 17901, 2702, 2959, 12603, 5016, 7666, 5890, 1276, 248, 35235, 1260, 8713, 667, 13261, 1806, 12373, 10436, 38234, 34798, 35788, 7809, 199, 21797, 33027, 1444, 2211, 3026, 31234, 624, 440, 32222, 3469, 3359, 217, 165, 1448, 34522, 5936, 10793, 17895, 10464, 2251, 10029, 33248, 33242, 2400, 2373, 16355, 32262, 33028, 37395, 2067, 3349, 37516, 7647, 476, 15775, 31262, 1137, 608, 30740, 3121, 39597, 5875, 77, 3220, 1470, 31678, 9744, 2089, 19211, 1812, 38042, 40807, 5262, 33274, 3399, 19919, 39560, 3001, 171, 31380, 22583, 35857, 8081, 1786, 1530, 6839, 2312, 19221, 30709, 532, 11827, 4903, 2683, 5, 12637, 405, 36198, 7858, 21821, 22745, 757, 38084, 1639

In [10]:
x = visual_embeddings[16668][0]
print(sum(x))

0.36309052142314613


In [12]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [13]:
# synopsis_embeddings_path = "./synopsis_autoencoder/50_synopsis_embeddings_10_epochs.pickle"
synopsis_embeddings_path = "./synopsis_autoencoder/sentence_embeddings_mpnet.pickle"
with open(synopsis_embeddings_path, 'rb') as f1:
  synopsis_embeddings = pickle.load(f1)
print(synopsis_embeddings.keys())

dict_keys([28891, 23273, 34599, 5114, 31758, 37510, 199, 38000, 35247, 2904, 32935, 34096, 15335, 918, 4181, 15417, 30276, 16664, 4672, 7588, 1253, 20767, 1425, 35363, 22673, 6862, 5680, 451, 9563, 30364, 33280, 36369, 27633, 31637, 38145, 552, 1689, 8609, 34944, 36214, 32863, 36160, 2265, 39940, 322, 38958, 4472, 97, 468, 7645, 1491, 19319, 1571, 35222, 38770, 269, 30123, 32998, 202, 5005, 18397, 3750, 33263, 32380, 35860, 3092, 38084, 37033, 28999, 388, 9724, 37985, 20057, 1172, 397, 2273, 4080, 11887, 254, 11135, 13119, 29755, 267, 3665, 37150, 12403, 39597, 15771, 82, 34745, 3782, 253, 12413, 5630, 38816, 29787, 2321, 4447, 1505, 1472, 9465, 35507, 38349, 12445, 1254, 35073, 597, 569, 19111, 14349, 4898, 249, 12893, 8142, 532, 21185, 890, 543, 949, 34504, 1729, 8310, 35075, 35678, 15809, 1366, 23225, 17909, 10521, 22319, 400, 9088, 33190, 20, 33221, 22097, 3014, 2890, 6702, 2035, 6351, 6377, 1485, 4163, 2143, 6007, 25517, 27947, 1695, 187, 14175, 35120, 779, 10012, 2605, 31973, 498

In [14]:
# import numpy as np
# synopsis_embeddings = {k: v.detach().numpy() for k, v in synopsis_embeddings.items()}

In [15]:
y = synopsis_embeddings[5114][0]
print(sum(y))
# y_norm = (y-np.min(y))/(np.max(y)-np.min(y))
# print(sum(y_norm))

-0.46975363883984755


In [16]:
visual_embeddings_keys = set(visual_embeddings.keys())
synopsis_embeddings_keys = set(synopsis_embeddings.keys())

print(visual_embeddings_keys - synopsis_embeddings_keys)
print(synopsis_embeddings_keys - visual_embeddings_keys)

visual_synopsis_embeddings_keys = visual_embeddings_keys.union(synopsis_embeddings_keys)
print(visual_synopsis_embeddings_keys)

{2916, 4934, 906, 1423, 5040, 4884, 1174, 2008, 94, 1342}
{38816, 39523, 35780, 39651, 39811, 14661, 33929, 2282, 1737, 30250, 38414, 31886, 40693, 38297, 37498, 39836, 22429, 39071}
{1, 32772, 5, 6, 7, 8, 15, 16, 32785, 18, 17, 20, 21, 22, 23, 32792, 24, 26, 25, 28, 29, 30, 31, 32, 32801, 33, 27, 32804, 43, 44, 32813, 46, 45, 48, 49, 50, 47, 52, 53, 54, 56, 57, 58, 59, 60, 32829, 62, 61, 63, 64, 32830, 67, 32836, 69, 68, 71, 72, 73, 74, 75, 32843, 77, 76, 79, 80, 81, 82, 83, 84, 85, 32854, 32852, 88, 89, 32851, 91, 92, 90, 94, 95, 96, 32863, 97, 32867, 99, 101, 102, 103, 104, 105, 32871, 32869, 108, 109, 32878, 110, 112, 113, 32881, 115, 116, 117, 32887, 120, 121, 122, 123, 124, 125, 32894, 126, 128, 127, 130, 131, 32900, 32901, 32902, 134, 136, 137, 138, 135, 32906, 132, 142, 143, 139, 145, 146, 147, 141, 149, 150, 151, 152, 153, 154, 155, 32924, 157, 158, 159, 156, 161, 162, 163, 164, 165, 32930, 32935, 32936, 32937, 169, 171, 167, 173, 170, 174, 168, 177, 178, 175, 180, 32949, 182,

In [65]:
# visual_embeddings[1][0]

In [66]:
# synopsis_embeddings[1][0]

In [23]:
visual_synopsis_embeddings = dict()
visual_temp = np.zeros(([1, 1000]))
synopsis_temp = np.zeros(([1, 768]))

for key in visual_synopsis_embeddings_keys:
  # print(key)
  if key in visual_embeddings and key in synopsis_embeddings:
    # print(len(visual_embeddings[key][0]), len(synopsis_embeddings[key][0]))
    visual_synopsis_embeddings[key] = np.concatenate((visual_embeddings[key][0], synopsis_embeddings[key][0]), axis = 0)
  elif key in visual_embeddings_keys:
    visual_synopsis_embeddings[key] = np.concatenate((visual_embeddings[key][0], synopsis_temp[0]), axis = 0)
  elif key in synopsis_embeddings_keys:
    visual_synopsis_embeddings[key] = np.concatenate((visual_temp[0], synopsis_embeddings[key][0]), axis = 0)

In [24]:
print(visual_synopsis_embeddings.keys())

dict_keys([1, 32772, 5, 6, 7, 8, 15, 16, 32785, 18, 17, 20, 21, 22, 23, 32792, 24, 26, 25, 28, 29, 30, 31, 32, 32801, 33, 27, 32804, 43, 44, 32813, 46, 45, 48, 49, 50, 47, 52, 53, 54, 56, 57, 58, 59, 60, 32829, 62, 61, 63, 64, 32830, 67, 32836, 69, 68, 71, 72, 73, 74, 75, 32843, 77, 76, 79, 80, 81, 82, 83, 84, 85, 32854, 32852, 88, 89, 32851, 91, 92, 90, 94, 95, 96, 32863, 97, 32867, 99, 101, 102, 103, 104, 105, 32871, 32869, 108, 109, 32878, 110, 112, 113, 32881, 115, 116, 117, 32887, 120, 121, 122, 123, 124, 125, 32894, 126, 128, 127, 130, 131, 32900, 32901, 32902, 134, 136, 137, 138, 135, 32906, 132, 142, 143, 139, 145, 146, 147, 141, 149, 150, 151, 152, 153, 154, 155, 32924, 157, 158, 159, 156, 161, 162, 163, 164, 165, 32930, 32935, 32936, 32937, 169, 171, 167, 173, 170, 174, 168, 177, 178, 175, 180, 32949, 182, 183, 32948, 185, 186, 184, 188, 189, 190, 191, 192, 193, 194, 32962, 196, 32961, 198, 199, 200, 201, 202, 203, 204, 205, 197, 207, 208, 209, 210, 32977, 212, 32981, 32979, 

In [26]:
visual_synopsis_embeddings = {k: v.reshape(1768) for k, v in visual_synopsis_embeddings.items()}

In [27]:
print(visual_synopsis_embeddings[5])

[-2.3561969  -1.9760169   0.27295205 ...  0.023355   -0.01295347
 -0.00901866]


In [28]:
idx_anime_id_dict = dict()
anime_id_idx_dict = dict()

for i, key in enumerate(visual_synopsis_embeddings):
  idx_anime_id_dict[i] = key
  anime_id_idx_dict[key] = i
print(idx_anime_id_dict)
print(anime_id_idx_dict)

{0: 1, 1: 32772, 2: 5, 3: 6, 4: 7, 5: 8, 6: 15, 7: 16, 8: 32785, 9: 18, 10: 17, 11: 20, 12: 21, 13: 22, 14: 23, 15: 32792, 16: 24, 17: 26, 18: 25, 19: 28, 20: 29, 21: 30, 22: 31, 23: 32, 24: 32801, 25: 33, 26: 27, 27: 32804, 28: 43, 29: 44, 30: 32813, 31: 46, 32: 45, 33: 48, 34: 49, 35: 50, 36: 47, 37: 52, 38: 53, 39: 54, 40: 56, 41: 57, 42: 58, 43: 59, 44: 60, 45: 32829, 46: 62, 47: 61, 48: 63, 49: 64, 50: 32830, 51: 67, 52: 32836, 53: 69, 54: 68, 55: 71, 56: 72, 57: 73, 58: 74, 59: 75, 60: 32843, 61: 77, 62: 76, 63: 79, 64: 80, 65: 81, 66: 82, 67: 83, 68: 84, 69: 85, 70: 32854, 71: 32852, 72: 88, 73: 89, 74: 32851, 75: 91, 76: 92, 77: 90, 78: 94, 79: 95, 80: 96, 81: 32863, 82: 97, 83: 32867, 84: 99, 85: 101, 86: 102, 87: 103, 88: 104, 89: 105, 90: 32871, 91: 32869, 92: 108, 93: 109, 94: 32878, 95: 110, 96: 112, 97: 113, 98: 32881, 99: 115, 100: 116, 101: 117, 102: 32887, 103: 120, 104: 121, 105: 122, 106: 123, 107: 124, 108: 125, 109: 32894, 110: 126, 111: 128, 112: 127, 113: 130, 11

In [29]:
profiles.head()

,profile,gender,favorites_anime,link
0,DesolatePsyche,Male,"['25013', '33352', '33674', '269', '8525', '45...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,"['918', '6956', '853', '34599', '11061', '3588...",https://myanimelist.net/profile/baekbeans
2,skrn,NaN,"['918', '17074', '32281', '2904', '9989', '194...",https://myanimelist.net/profile/skrn
3,edgewalker00,Male,"['5680', '37349', '2904', '849', '3588']",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,"['9617', '5680', '4181', '235', '31646', '4382...",https://myanimelist.net/profile/aManOfCulture99


In [30]:
def clean_favorites_anime(favorites_anime):
  """
  Coinverts the str(list) -> list format
  """
  favorites_anime = favorites_anime[1:-1].strip().split(',')
  favorites_anime = list(set(anime.strip()[1:-1] for anime in favorites_anime))
  return favorites_anime

In [31]:
profiles['favorites_anime'] = profiles.apply(lambda x: clean_favorites_anime(x['favorites_anime']), axis = 1)
profiles.head()

,profile,gender,favorites_anime,link
0,DesolatePsyche,Male,"[25013, 269, 226, 34618, 28851, 17074, 2904, 1...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,"[34599, 13601, 6956, 31964, 11061, 9253, 3588,...",https://myanimelist.net/profile/baekbeans
2,skrn,NaN,"[512, 9989, 32281, 17074, 2904, 11741, 1943, 9...",https://myanimelist.net/profile/skrn
3,edgewalker00,Male,"[849, 5680, 2904, 3588, 37349]",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,"[849, 5680, 4382, 35180, 31646, 235, 4181, 961...",https://myanimelist.net/profile/aManOfCulture99


In [32]:
# temp_favorite_animes = profiles[profiles.profile == 'DesolatePsyche']['favorites_anime']
# print(temp_favorite_animes[0])

In [33]:
# temp = reviews.loc[reviews.profile == 'DesolatePsyche']
# temp2 = temp.loc[temp.anime_uid.isin([int(favorite_anime) for favorite_anime in temp_favorite_animes[0]])]
# if 34618 in list(temp2.anime_uid):
#   print(list(temp2[temp2.anime_uid == 34618].score)[0])

In [34]:
# list(temp2.anime_uid)

In [35]:
train_df.head()

,uid,profile,anime_uid,text,score,scores,link,old_profile,old_anime_uid
0,255938,0,0,"first things first. my ""reviews"" system is exp...",8,"{'Sound': '10', 'Overall': '8', 'Animation': '...",https://myanimelist.net/reviews.php?id=255938,DesolatePsyche,34096
2,253664,2,2,"art 9/10: it is great, especially the actions ...",7,"{'Story': '7', 'Sound': '8', 'Overall': '7', '...",https://myanimelist.net/reviews.php?id=253664,skrn,28891
3,8254,3,3,taking place 1 yr from where season 1 trailed ...,9,"{'Overall': '9', 'Story': '9', 'Enjoyment': '9...",https://myanimelist.net/reviews.php?id=8254,edgewalker00,2904
4,291149,4,4,kyoto animations greatest strength is being ab...,10,"{'Story': '10', 'Sound': '9', 'Overall': '10',...",https://myanimelist.net/reviews.php?id=291149,aManOfCulture99,4181
5,10046,5,3,when i first saw code geass i was like omgzxwt...,10,"{'Story': '9', 'Animation': '10', 'Sound': '10...",https://myanimelist.net/reviews.php?id=10046,eneri,2904


In [36]:
profiles['rated_animes'] = profiles.apply(lambda x: list((train_df[train_df.old_profile == x['profile']]['old_anime_uid'])), axis = 1)
profiles.head()

,profile,gender,favorites_anime,link,rated_animes
0,DesolatePsyche,Male,"[25013, 269, 226, 34618, 28851, 17074, 2904, 1...",https://myanimelist.net/profile/DesolatePsyche,"[34096, 12403, 23225, 2035, 31173, 31240, 3181..."
1,baekbeans,Female,"[34599, 13601, 6956, 31964, 11061, 9253, 3588,...",https://myanimelist.net/profile/baekbeans,"[35507, 3588, 853, 10408, 9253, 33674, 9989, 2..."
2,skrn,NaN,"[512, 9989, 32281, 17074, 2904, 11741, 1943, 9...",https://myanimelist.net/profile/skrn,"[28891, 4107, 32935, 4081, 4565, 20583, 1535]"
3,edgewalker00,Male,"[849, 5680, 2904, 3588, 37349]",https://myanimelist.net/profile/edgewalker00,"[2904, 2251, 3358, 770, 3467]"
4,aManOfCulture99,Male,"[849, 5680, 4382, 35180, 31646, 235, 4181, 961...",https://myanimelist.net/profile/aManOfCulture99,[4181]


In [37]:
def getWeightedUserProfile(profile_id, favorite_animes, rated_animes):
  # print(profile_id, favorite_animes)
  # print(len(favorite_animes), len(rated_animes))
  all_animes = list(set(favorite_animes).union(set(rated_animes)) - {''})
  # print(len(all_animes))
  temp = train_df.loc[train_df.old_profile == profile_id]
  temp2 = temp.loc[temp.old_anime_uid.isin([int(anm) for anm in all_animes])]
  # print(temp2.head())
  weighted_user_profile = np.zeros(([1768]))
  cnt = 0
  for anime in all_animes:
    if anime == '':
      continue
    if int(anime) in visual_synopsis_embeddings:
      multiplier = 10
      if int(anime) in list(temp2.old_anime_uid):
        multiplier = list(temp2[temp2.old_anime_uid == int(anime)].score)[0]
        # print(multiplier)
      cnt += multiplier
      weighted_user_profile = np.add(weighted_user_profile, multiplier*visual_synopsis_embeddings[int(anime)])
  if cnt != 0:
    weighted_user_profile = weighted_user_profile/cnt
  # print(cnt, len(favorite_animes))
  # print(len(weighted_user_profile))
  # print(weighted_user_profile)
  return weighted_user_profile

In [26]:
# print(getWeightedUserProfile('DesolatePsyche', temp_favorite_animes[0]))

In [38]:
profiles['weighted_user_profile'] = profiles.apply(lambda x: getWeightedUserProfile(x['profile'], x['favorites_anime'], x['rated_animes']), axis = 1)

In [39]:
profiles.head()

,profile,gender,favorites_anime,link,rated_animes,weighted_user_profile
0,DesolatePsyche,Male,"[25013, 269, 226, 34618, 28851, 17074, 2904, 1...",https://myanimelist.net/profile/DesolatePsyche,"[34096, 12403, 23225, 2035, 31173, 31240, 3181...","[-1.15137858161353, -1.1582384183677523, -0.27..."
1,baekbeans,Female,"[34599, 13601, 6956, 31964, 11061, 9253, 3588,...",https://myanimelist.net/profile/baekbeans,"[35507, 3588, 853, 10408, 9253, 33674, 9989, 2...","[-1.5401537639578593, -1.6263397157806712, -0...."
2,skrn,NaN,"[512, 9989, 32281, 17074, 2904, 11741, 1943, 9...",https://myanimelist.net/profile/skrn,"[28891, 4107, 32935, 4081, 4565, 20583, 1535]","[-1.146176910918692, -0.6639952386958875, 0.02..."
3,edgewalker00,Male,"[849, 5680, 2904, 3588, 37349]",https://myanimelist.net/profile/edgewalker00,"[2904, 2251, 3358, 770, 3467]","[-1.3465011408040812, -1.4808536702459985, 0.0..."
4,aManOfCulture99,Male,"[849, 5680, 4382, 35180, 31646, 235, 4181, 961...",https://myanimelist.net/profile/aManOfCulture99,[4181],"[-1.3235332012176513, -0.9343860604546287, -0...."


In [40]:
user_profile_dict = pd.Series(profiles.weighted_user_profile.values,index=profiles.profile).to_dict()
print(user_profile_dict.keys())

dict_keys(['DesolatePsyche', 'baekbeans', 'skrn', 'edgewalker00', 'aManOfCulture99', 'eneri', 'Waffle_Empress', 'jchang', 'angelsreview', 'CalebTheMenace', 'Kiethol', 'Eanki', '12sed', 'OVERPOWERED99', 'Kaishuu', 'Rigas', 'Leyvon', 'Sime3134', 'Itaku', 'SieghartTheMage', 'ggultra2764', '0Seru0', 'BowlingJD', 'gwern', 'greatyu', 'mattpointon77', 'yolerMit', 'SamuraiSteve', 'kyojin13', 'HybridMedia', 'eyerok', 'patrickRos', 'shadowfireflame', 'SerpentineJ', 'tinypinkghostie', 'stonnyfor', 'BabyGirl06301', 'ToastytheBaker', 'ImTheRhetorician', 'Eoussama', 'Aphyxia', 'ParaParaJMo', 'ChouEritto', 'Yakarin', 'Paulwillson26', 'Eneko-chan', 'Lima-Sama', 'Baconzombie', 'skankfish', 'AlexJetski', 'IceAndCream', 'Jolon', 'Seabury', 'Fleure', 'Jodyqt', 'Lindle', 'ZephSilver', 'Mimi_Taylor', 'Detective', 'theillusionary', 'Jakerams', 'showandAG', 'xScHeiZe', 'ChildHatchling', 'Lohuydahutt', 'TabiNoTochuu', 'LoveArrowShooto', 'For_Reign', 'vanmarckestevy', 'BTDee', 'Asasphinx', 'LegendAqua', 'CinaGi

In [41]:
user_profile_dict['DesolatePsyche']

array([-1.15137858, -1.15823842, -0.27751524, ..., -0.00368601,
        0.02151995, -0.03114039])

In [64]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import math

mse = 0
cnt = 0
for k in list(user_profile_dict.keys()):
  k_profile = user_profile_dict[k].reshape(1,len(user_profile_dict[k]))
  test_k = test_mat[user_old2new_id_dict[k]]
  to_predict = np.where(test_k > 0)[0]
  if len(to_predict) == 0:
    continue
  rating_true = test_k[to_predict]
  to_predict_old = [movie_new2old_id_dict[anime_uid] for anime_uid in to_predict]
  predicted = [cosine_similarity(k_profile, visual_synopsis_embeddings[anime_uid].reshape(1, 1768)) for anime_uid in to_predict_old]
  predicted = [pred_sim[0][0]*10 for pred_sim in predicted]

  mse += mean_squared_error(rating_true, predicted)
  cnt += 1

# print(mse, cnt)
rmse = math.sqrt(mse/cnt)
print(rmse)

2.5606178482195174
